# Packages

In [1]:
import pandas as pd 
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Nasconde i FutureWarning

# Per aggiungere un path per dei pacchetti selfmade
import sys
sys.path.append('.../__HandmadePackages__')
sys.path.append('.../EvaluateKeyGenes')
# Classe per agevolare la valutazione della performance di un subset di keygenes
from EvaluatePipeline import Evaluate

# Funzioni selfmade per preprocessing
import preprocessing

# Classe selfmade per creare report LaTeX
from LaTeX import LaTeX

## Import the data

In [2]:
# Inizializzare report LaTeX
doc = LaTeX(formato="V",
            titolo_report="Performance key genes con: Train GSE68086,  Test GSE156902",
            indice=True,
            autore="Mattia Manna",
            data="\\today",
            geometry="left=1cm,bottom=1.5cm"
)


In [ ]:
path_data = ".../Data product"
conditions = ["DEGS","HUBS","BETWEENNESS","CLOSENESS","CLOSENESS_last5","EIGENVECTOR","LCC","ElasticNet"]


for condition in tqdm(conditions):

   
   doc.add_section(f"Classificazione con {condition}") # Per il report pdf
   doc.add_subsection(f"Informazioni sulla numerosità dei dataset") # Per il report pdf


   df_train = pd.read_csv(f"{path_data}/normalizedcounts_trained_onGSE68086_GBM_HC_{condition}.csv")
   #df_train = pd.read_csv(f"{path_data}/normalizedcounts_trained_onGSE68086_GBM_HC_{condition}_ONLYcommongenes.csv")
   #print(f"df_train shape: {df_train.shape}, Numerosità classi: {dict(pd.value_counts(df_train['condition']))}")
   doc.add_text(f"Numerosità GSE68086 (train) all'importazione: {df_train.shape}\\\\") # Per il report pdf
   doc.add_text(f"Numerosità classi: {dict(pd.value_counts(df_train['condition']))}") # Per il report pdf

   doc.newline_emptyline() # Per il report pdf

   df_test1 = pd.read_csv(f"{path_data}/testdata_normalized_GSE156902_GBM_HC.csv")
   #print(f"df_test1 shape: {df_test1.shape}, Numerosità classi: {dict(pd.value_counts(df_test1['condition']))}\n")
   doc.add_text(f"Numerosità GSE156902 (test) all'importazione: {df_test1.shape}\\\\") # Per il report pdf
   doc.add_text(f"Numerosità classi: {dict(pd.value_counts(df_test1['condition']))}") # Per il report pdf

   doc.newline_emptyline() # Per il report pdf

   # Cleaning train dataset 
   df_train.rename(columns={"Unnamed: 0":"patient"},inplace=True)
   df_train.set_index("patient",inplace=True)
   cleaning_output_df_train = preprocessing.feature_to_numeric(dataset=df_train,
                                                               feature_name="condition",
                                                               verbose=False)
   df_train = cleaning_output_df_train["dataset"].copy()
   #print(f'Train labels: {cleaning_output_df_train["labels"]["condition"]}')
   #----#
   # Cleaning test1 dataset 
   df_test1.rename(columns={"Unnamed: 0":"patient"},inplace=True)
   df_test1.set_index("patient",inplace=True)
   cleaning_output_df_test1 = preprocessing.feature_to_numeric(dataset=df_test1,
                                                               feature_name="condition",
                                                               verbose=False)
   df_test1 = cleaning_output_df_test1["dataset"].copy()
   #print(f'Test labels: {cleaning_output_df_test1["labels"]["condition"]}\n')

   # Definire l'oggetto Evaluate
   gbm_subset1_evaluation = Evaluate(train_data=df_train,
                                    test_data=df_test1,
                                    seed=0,
                                    target_feature_name="condition",
                                    labels = cleaning_output_df_test1["labels"]["condition"],
                                    verbose=False
                                    )
   
   # Training e testing dei modelli
   gbm_subset1_evaluation.LogisticRegression(verbose=False,train=True) # Prova anche il modello con LeaveOneOut su dati di train
   gbm_subset1_evaluation.SupportVectorClassifier(verbose=False,train=True)
   gbm_subset1_evaluation.SupportVectorClassifierKFold(metric_to_optimize="recall",
                                                      class_to_optimize=0,
                                                      stratify=True,
                                                      fold=5,
                                                      param_grid= { 'C': [0.01,0.05,0.1, 1, 10, 100],   
                                                                     'kernel': ['linear', 'rbf'],
                                                                     'gamma': ['scale', 'auto']},
                                                      verbose=False
                                                      )
   gbm_subset1_evaluation.RandomForest(verbose=False,train=True)
   gbm_subset1_evaluation.KNearestNeighbors(verbose=False,train=True)
   gbm_subset1_evaluation.NaiveBayesBinomial(verbose=False,train=True)
   gbm_subset1_evaluation.NaiveBayesGaussian(verbose=False,train=True)


   # Performance dei modelli
   risultati = gbm_subset1_evaluation.get_performance(verbose=False) # non funziona il suggeritore

   # Compilazione report LaTeX
   doc.add_text(f"Numerosità GSE68086 (train): {gbm_subset1_evaluation.train_data_shape}. Ci sono {gbm_subset1_evaluation.train_data_shape[1]-1} geni ed una variabile target.\\\\") # Per il report pdf
   doc.add_text(f"Numerosità GSE156902 (test): {gbm_subset1_evaluation.test_data_shape}. Ci sono {gbm_subset1_evaluation.test_data_shape[1]-1} geni ed una variabile target.\\\\") # Per il report pdf
   doc.newline_emptyline() # Per il report pdf
   doc.add_text(f"Numero di geni comuni tra i due dataset: {len(gbm_subset1_evaluation.common_genes)}\\\\")# Per il report pdf
   doc.add_text(f"Seed utilizzato per la classificazione: {gbm_subset1_evaluation.seed}")# Per il report pdf

   doc.add_subsection("Risultati performance") # Per il report pdf
   
   # Logistic regressiom
   doc.add_subsubsection("Performance tramite Logistic Regression (LR)") # Per il report pdf
   doc.add_table(risultati["LR"]["df_report_cm"],include_index=True,intestazione_tabella="LR") # Per il report pdf
   doc.add_table(risultati["LR"]["confusion_matrix"],include_index=True,intestazione_tabella="Confusion Matrix LR") # Per il rep

   # Support vector classifier
   doc.add_subsubsection("Performance tramite Support Vector Classifier (SVC)") # Per il report pdf
   doc.add_table(risultati["SVC"]["df_report_cm"],include_index=True,intestazione_tabella="SVC") # Per il report pdf
   doc.add_table(risultati["SVC"]["confusion_matrix"],include_index=True,intestazione_tabella="Confusion Matrix SVC") # Per il rep
   doc.add_newpage()    

   # Support vector classifier KFold
   doc.add_subsubsection("Performance tramite Support Vector Classifier (SVC) con KFold, k=5") # Per il report pdf
   doc.add_table(risultati["SVC_KF"]["df_report_cm"],include_index=True,intestazione_tabella="SVC KFold, k = 5") # Per il rep
   doc.add_table(risultati["SVC_KF"]["confusion_matrix"],include_index=True,intestazione_tabella="Confusion Matrix SVC KFold") # Per il rep
   
   
   # RandomForest classifier
   doc.add_subsubsection("Performance tramite Random Forest Classifier") # Per il report pdf
   doc.add_table(risultati["RF"]["df_report_cm"],include_index=True,intestazione_tabella="RF") # Per il rep
   doc.add_table(risultati["RF"]["confusion_matrix"],include_index=True,intestazione_tabella="Confusion Matrix RF") # Per il rep
 

   #K Nearest Neighbors classifier
   doc.add_subsubsection("Performance tramite K Nearest Neighbors Classifier") # Per il report pdf
   doc.add_table(risultati["KNN"]["df_report_cm"],include_index=True,intestazione_tabella="KNN") # Per il rep
   doc.add_table(risultati["KNN"]["confusion_matrix"],include_index=True,intestazione_tabella="Confusion Matrix KNN") # Per il rep
   doc.add_newpage()    

   #Naive Bayes Binomial classifier
   doc.add_subsubsection("Performance tramite Naive Bayes Binomial Classifier") # Per il report pdf
   doc.add_table(risultati["NB_Binomial"]["df_report_cm"],include_index=True,intestazione_tabella="NB Binomial") # Per il rep
   doc.add_table(risultati["NB_Binomial"]["confusion_matrix"],include_index=True,intestazione_tabella="Confusion Matrix NB Binomial") # Per il rep

   #Naive Bayes Gaussian classifier
   doc.add_subsubsection("Performance tramite Naive Bayes Gaussian Classifier") # Per il report pdf
   doc.add_table(risultati["NB_Gaussian"]["df_report_cm"],include_index=True,intestazione_tabella="NB Gaussian") # Per il rep
   doc.add_table(risultati["NB_Gaussian"]["confusion_matrix"],include_index=True,intestazione_tabella="Confusion Matrix NB Gaussian") # Per il rep


   

   doc.add_newpage()    
   doc.add_subsection(f"Summary {condition}") # Per il rep
   
   doc.add_subsubsection(f"Summary {condition} train data") # Per il rep
   doc.add_table(gbm_subset1_evaluation.performance_summary_train(condition="GBM"),
                  include_index=True,
                  intestazione_tabella="Riassunto performance train, condizione di riferimento GBM",
                  separatore_righe=True,
                  colore_righe=True) # Per il rep 


   
   doc.add_subsubsection(f"Summary {condition} test data")   # Per il rep
   doc.add_table(gbm_subset1_evaluation.performance_summary(condition="GBM"),
               include_index=True,
               intestazione_tabella="Riassunto performance test, condizione di riferimento GBM",
               separatore_righe=True,
               colore_righe=True) # Per il rep
   
   doc.add_newpage()    


100%|██████████| 8/8 [01:18<00:00,  9.76s/it]


In [4]:
doc.export(".../Train_GSE68086_Test_GSE156902.tex",knit=True)

File Train_GSE68086_Test_GSE156902.tex già presente, salvato con nome: Train_GSE68086_Test_GSE156902_2.tex
